# (D3) solution with 10=4+6 split

The strategy here should work with any generic solution to the Einstein equation, so let's try to do it with the D3 brane solution in 10d

$$
ds_{10,E}^2 = H^{a} ds_{1,3}^2 + H^{b} ds_6^2
$$

The solution is balanced with a $F_5 = *_{10} F_5$ and $a=-\tfrac12, b = \tfrac12$

Explicit derivative computations have to be done with sp.Function, after that's done it's more convenient to swap in $H, H', H'', r ,r^2$ as sp.symbols

# Gravity set up

In [1]:
import sympy as sp
from helpers.form_helpers import *
from helpers.einstein_helpers import *

In [2]:
# coordinates
x = sp.symbols('x0:4', real=True)
y = sp.symbols('y0:6', real=True)
coords = (*x,*y)
n = len(coords)

# Parameters and symbols
apar, bpar = sp.symbols('a b', real=True)
r_sym = sp.symbols('r', real=True, positive=True)

# expressions for radial coordinate r and r**2
r2 = sum(y_i**2 for y_i in y)
r = sp.sqrt(r2)

# harmonic function H, both as function of r, and as symbols
H = sp.Function('H')(r)
H_, H_p, H_pp = sp.symbols("H H' H''")
HARMONIC_DIMENSIONS = 6


print(f"We are in {n} dimensions, with coordinates {coords}")
print(f"Radial coordinate r = {r}, r^2 = {r2}")
print(f"Harmonic function H is harmonic in {HARMONIC_DIMENSIONS} dimensions.")

We are in 10 dimensions, with coordinates (x0, x1, x2, x3, y0, y1, y2, y3, y4, y5)
Radial coordinate r = sqrt(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2), r^2 = y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2
Harmonic function H is harmonic in 6 dimensions.


In [3]:
# metric
g_MN = sp.zeros(n)
g_MN[0:4,0:4] = H**apar * sp.diag(-1, 1, 1, 1)
g_MN[4:n,4:n] = H**bpar * sp.eye(6)
g = sp.simplify(sp.sqrt(-(sp.det(g_MN))))
g_MN = sp.simplify(g_MN)

inv_g_MN = g_MN.inv()

In [4]:
# standard substitution procedure for later simplifications
def standard_subs(expr):
    """
    Substitutes sympy derivatives on function H(r) with symbols H, H', H''
    Then imposes the harmonic condition on H(r)

    Should be applied after derivatives have been computed
    """
    expr = sp.simplify(expr)
    expr = sp.simplify(expr.subs({H:H_,
                                  sp.sqrt(r2): r_sym,}))
    expr = sp.simplify(expr.subs({sp.Derivative(sp.Function('H')(r_sym), r_sym): H_p,
                                  sp.Derivative(sp.Function('H')(r_sym), (r_sym, 2)): H_pp}))
    expr = sp.simplify(impose_harmonic_condition_sym(expr, HARMONIC_DIMENSIONS, H_pp, H_p, r_sym))
    return expr

In [5]:
g_MN

Matrix([
[-H(sqrt(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2))**a,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0],
[                                                         0, H(sqrt(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2))**a,                                                         0,                                                         0,                                                         0,                                                         0,                                           

# Computing Ricci Tensor

In [6]:
R_MN = ricci_tensor(g_MN, coords)

Computed Gamma^0_...,Computed Gamma^1_...,Computed Gamma^2_...,Computed Gamma^3_...,Computed Gamma^4_...,Computed Gamma^5_...,Computed Gamma^6_...,Computed Gamma^7_...,Computed Gamma^8_...,Computed Gamma^9_...,Computed R[0,...],Computed R[1,...],Computed R[2,...],Computed R[3,...],Computed R[4,...],Computed R[5,...],Computed R[6,...],Computed R[7,...],Computed R[8,...],Computed R[9,...],

## Look at some Ricci Tensor components

In [7]:
sp.simplify(standard_subs(R_MN[0,0]))

H**(a - b - 2)*H'**2*a*(2*a*y0**2 + 2*a*y1**2 + 2*a*y2**2 + 2*a*y3**2 + 2*a*y4**2 + 2*a*y5**2 + 2*b*y0**2 + 2*b*y1**2 + 2*b*y2**2 + 2*b*y3**2 + 2*b*y4**2 + 2*b*y5**2 - y0**2 - y1**2 - y2**2 - y3**2 - y4**2 - y5**2)/(2*r**2)

In [8]:
expr = R_MN[4,4]
expr = standard_subs(expr)
expr = sp.simplify(expr.subs({apar: -sp.Rational(1,2), bpar: sp.Rational(1,2)}))
expr

H'*(-6*H*r**2 + 6*H*(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2) + H'*r*(-y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2))/(4*H**2*r**3)

# Form Fields

For an $n$ form, the Einstein equation is

$$
R_{MN} = S_{MN} = \frac{1}{2}
\frac{\lambda_n}{(n-1)!}(F_n)_{M ...}(F_n)_N{}^{...}
-\frac{1}{10}\frac{1}{2}\frac{3\lambda_n}{n!}F_n^2 g_{MN}
$$

We would like to try with the 5-form that is self-dual

$$
(F_5)_{m\mu_0\mu_1\mu_2\mu_3} = c_1 \partial_m (H^{-1}) \epsilon_{\mu_0\mu_1\mu_2\mu_3},\quad
(F_5)_{m m_1 m_2 m_3 m_4} = c_2 \epsilon_{m m_1 m_2 m_3 m_4 p} \delta^{pq}\partial_q H
= c_2 \epsilon_{m m_1 m_2 m_3 m_4 p} \partial_p H
$$

## define form field F

In [9]:
# from itertools import product, permutations
# from sympy.combinatorics.permutations import Permutation

# def epsilon(n, idx):
#     """
#     Returns the Levi-Civita symbol in the given number of dimensions with the specified indices.
#     The indices should be a tuple or list of integers.

#     Examples:
#     >>> epsilon(3, [0, 1, 2]) # returns 1
#     >>> epsilon(3, [1, 0, 2]) # returns -1
#     >>> epsilon(3, [0, 2, 2]) # returns 0
#     """
#     if len(idx) != n:
#         raise ValueError("Number of indices must match the number of dimensions.")
#     if any(i < 0 or i >= n for i in idx):
#         raise ValueError("Indices must be within the valid range.")
#     if len(set(idx)) < n:
#         return 0  # repeated index → 0

#     # parity via inversion count
#     inv = 0
#     a = tuple(idx)
#     for i in range(n - 1):
#         ai = a[i]
#         inv += sum(ai > a[j] for j in range(i + 1, n))
#     return -1 if (inv & 1) else 1


# def perm_with_sign(t):
#     """
#     Generate all permutations of the input tuple `t` along with their sign.

#     Usage:
#     t = (1, 2, 3, 11)
#     for tup, sgn in perm_with_sign(t):
#         print(tup, sgn)
#     """
#     n = len(t)
#     for p in permutations(range(n)):                # p is a perm of indices
#         yield tuple(t[i] for i in p), Permutation(p).signature()

# def find_nonzero_indices(_tensor):
#     """
#     Given a sympy tensor F, find all indices where F is nonzero.
#     Returns a list of index tuples.
#     """
#     nonzeros = []
#     for idx in product(*(range(s) for s in _tensor.shape)):
#         val = _tensor[idx]
#         if val != 0:                    # fast, but purely syntactic
#             nonzeros.append(idx)
#     return nonzeros

# def antisymmetrize_tensor(__tensor):
#     """
#     Given a sympy tensor F, antisymmetrize it by filling in all permutations
#     of its nonzero components with the appropriate sign.
#     """
#     nonzeros = find_nonzero_indices(__tensor)

#     # iterate over the nonzero indices, apply antisymmetry
#     for nonzero_index in nonzeros:
#         for permuted_indices, sgn in perm_with_sign(nonzero_index):
#             __tensor[permuted_indices] = sgn * __tensor[nonzero_index]
#     return __tensor

# def get_independent_form_indices(num_dimensions, rank):
#     """
#     Get a list of independent index combinations for an antisymmetric form
#     of given rank in the specified number of dimensions.

#     Example:
#     >>> get_independent_form_indices(4, 2)
#     [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
#     """
#     from itertools import combinations
#     return list(combinations(range(num_dimensions), rank))

In [10]:
# define non-zero components of form field F
F = sp.MutableDenseNDimArray.zeros(*([n] * 5))
c_1, c_2 = sp.symbols('c_1 c_2')
d = sp.symbols('d')

for i in range(6):
    F[i+4,0,1,2,3] = c_1 * sp.diff(H**(-1), y[i])

F[4,5,6,7,8] = c_2 * sp.diff(H, coords[9])
F[5,6,7,8,9] = -c_2 * sp.diff(H, coords[4])
F[6,7,8,9,4] = c_2 * sp.diff(H, coords[5])
F[7,8,9,4,5] = -c_2 * sp.diff(H, coords[6])
F[8,9,4,5,6] = c_2 * sp.diff(H, coords[7])
F[9,4,5,6,7] = -c_2 * sp.diff(H, coords[8])

# get all components via antisymmetrization
F = antisymmetrize_tensor(F)

## Compute Hodge dual

In [11]:
# def hodge_dual(tensor, metric, signature=-1):
#     """
#     we assume the metric is diagonal and that tensor is a form field
#     """
#     abs_det_g = signature * sp.det(metric)
#     sqrt_abs_det_g = sp.sqrt(abs_det_g)

#     rank = len(tensor.shape)
#     num_dims = metric.shape[0]
#     dual_rank = num_dims - rank
#     dual_tensor = sp.MutableDenseNDimArray.zeros(*([num_dims] * dual_rank))

#     # find independent components of the dual tensor, upto antisymmetry
#     independent_dual_indices = get_independent_form_indices(num_dims, dual_rank)

#     for dual_idx in independent_dual_indices:
#         remaining_indices = [i for i in range(num_dims) if i not in dual_idx]
#         # print(dual_idx,remaining_indices)

#         # sum over all permutations of the remaining indices
#         running_sum = sp.S(0)
#         remaining_indices_perms = permutations(remaining_indices)
#         for remaining_perm in remaining_indices_perms:
#             # print(dual_idx + remaining_perm)

#             contribution = sqrt_abs_det_g * epsilon(num_dims, dual_idx + remaining_perm) * F[remaining_perm]
#             for summed_index in remaining_perm:
#                 # raise index using inverse metric
#                 contribution *= inv_g_MN[summed_index, summed_index]
#             running_sum += contribution

#         dual_tensor[dual_idx] = running_sum / sp.factorial(rank)

#     return dual_tensor

# F_dual = hodge_dual(F, g_MN)

## check self-duality at $c_1 = c_2 = 1$

In [12]:
F_dual = hodge_dual(F, g_MN)
for independent_index in get_independent_form_indices(10,5):
    # print(independent_index)
    # print(standard_subs(F_dual[independent_index]).subs({apar: sp.Rational(-1,2), bpar: sp.Rational(1,2)}))
    print(standard_subs(F_dual[independent_index]).subs({apar: sp.Rational(-1,2), bpar: sp.Rational(1,2), c_1:1, c_2:1}) == standard_subs(F[independent_index]).subs({apar: sp.Rational(-1,2), bpar: sp.Rational(1,2), c_1:1, c_2:1}))


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Compute $F^2$

In [13]:
def compute_form_squared(form_field, inv_metric):
    """
    Compute the squared norm of a form field using the inverse metric.
    Assumes the form field is fully antisymmetric, and the metric is diagonal
    """
    num_dims = inv_metric.shape[0]
    rank = len(form_field.shape)
    
    print(f"Computing squared norm of a rank {rank} form in {num_dims} dimensions.")

    squared_norm = sp.S(0)

    # # Get independent components of the tensor
    independent_indices = get_independent_form_indices(num_dims, rank)

    for idx in independent_indices:
        component = form_field[idx]
        if component != 0:
            contribution = component**2
            for i in idx:
                contribution *= inv_metric[i, i]
            squared_norm += contribution * sp.factorial(rank)

    return squared_norm

def compute_FF_MN(form_field, inv_metric):
    """
    Compute the tensor F_{MPQRS} F_N^{PQRS}
    Assumes the form field is fully antisymmetric, and the metric is diagonal
    """
    num_dims = inv_metric.shape[0]
    rank = len(form_field.shape)
    
    print(f"Computing F_M PQRS F_N^PQRS for a rank {rank} form in {num_dims} dimensions.")

    toret = sp.MutableDenseNDimArray.zeros(n, n)
    
    # we are essentially summing over a worth of rank 4 antismmetric indices
    independent_sub_indices = get_independent_form_indices(n, 4)

    for M in range(n):
        for N in range(n):
            sum_term = sp.S(0)
            for idx in independent_sub_indices:
                if M in idx or N in idx: # vanishes due to antisymmetry
                    continue
                component_M = form_field[(M,) + idx]
                component_N = form_field[(N,) + idx]
                contribution = component_M * component_N
                for i in idx:
                    contribution *= inv_metric[i, i]
                sum_term += contribution * sp.factorial(4)
            toret[M, N] = sp.simplify(sum_term)

    return toret

def compute_S_MN(metric, rank, ff_MN, ff):
    """
    Given FF_{MN} and F^2, compute S_{MN} as defined in eq 2.2b of hep-th/9701088
    """
    n_dims = metric.shape[0]
    toret = sp.MutableDenseNDimArray.zeros(n_dims, n_dims)

    for M in range(n_dims):
        for N in range(n_dims):
            toret[M, N] = sp.Rational(1,2 * sp.factorial(rank - 1) ) * (ff_MN[M, N] - sp.Rational(rank-1,rank * (n_dims-1)) * metric[M, N] * ff)
    return toret

In [14]:
# S_MN = sp.MutableDenseNDimArray.zeros(n, n)
# for M in range(n):
#     for N in range(n):
#         S_MN[M,N] = sp.Rational(1,2) * sp.Rational(1,sp.factorial(3)) * FF_MN[M,N] - sp.Rational(1,10) * sp.Rational(1,2) * sp.Rational(3,sp.factorial(4)) * g_MN[M,N] * Fsquared
#         S_MN[M,N] = sp.simplify(S_MN[M,N])

In [15]:
S_MN = compute_S_MN(g_MN, 5, compute_FF_MN(F, inv_g_MN), compute_form_squared(F, inv_g_MN))

Computing F_M PQRS F_N^PQRS for a rank 5 form in 10 dimensions.
Computing squared norm of a rank 5 form in 10 dimensions.


In [16]:
sp.simplify(standard_subs(S_MN[0,0]))

H**(-3*a - 6*b - 4)*H'**2*(5*H**(5*b)*c_1**2 + 4*H**(4*a + b + 4)*c_2**2)/18

In [17]:
sp.simplify(standard_subs(compute_form_squared(F, inv_g_MN)))

Computing squared norm of a rank 5 form in 10 dimensions.


120*H**(-4*a - 6*b - 4)*H'**2*(-H**(5*b)*c_1**2 + H**(4*a + b + 4)*c_2**2)

In [18]:
Fsquared = sp.S(0)

for idx1 in range(n):
    for idx2 in range(n):
        for idx3 in range(n):
            for idx4 in range(n):
                for idx5 in range(n):
                    F_component = F[idx1, idx2, idx3, idx4, idx5]
                    Fsquared += F_component**2 * inv_g_MN[idx1,idx1] * inv_g_MN[idx2,idx2] * inv_g_MN[idx3,idx3] * inv_g_MN[idx4,idx4] * inv_g_MN[idx5,idx5]

Fsquared = sp.simplify(standard_subs(Fsquared))
Fsquared

120*H**(-4*a - 6*b - 4)*H'**2*(-H**(5*b)*c_1**2 + H**(4*a + b + 4)*c_2**2)

### Compute $(FF)_{MN}$

In [19]:
FF_MN = compute_FF_MN(F, inv_g_MN)

Computing F_M PQRS F_N^PQRS for a rank 5 form in 10 dimensions.


In [20]:
sp.simplify(standard_subs(FF_MN[6,6]))

24*H**(-4*a - 4*b - 4)*H'**2*(-H**(4*b)*c_1**2*y2**2 + H**(4*a + 4)*c_2**2*(y0**2 + y1**2 + y3**2 + y4**2 + y5**2))/r**2

### Compute $S_{MN}$

## Gather the equations (and simplify them)

$R_{00} = S_{00}$ and its simplification

In [21]:
eq00 = (sp.Eq(S_MN[0,0],R_MN[0,0])) #.subs({apar: -sp.Rational(1,2), bpar: sp.Rational(1,2)})
eq00 = standard_subs(eq00)
eq00 = sp.simplify(eq00.subs({apar: -sp.Rational(1,2), bpar: sp.Rational(1,2)}))
eq00

Eq(H'**2*(5*c_1**2 + 4*c_2**2)/(18*H**3), H'**2*(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2)/(4*H**3*r**2))

$R_{44} = S_{44}$ and its simplification

In [22]:
eq44 = (sp.Eq(S_MN[4,4],R_MN[4,4])) #.subs({apar: -sp.Rational(1,2), bpar: sp.Rational(1,2)})
eq44 = standard_subs(eq44)
eq44 = sp.simplify(eq44.subs({apar: -sp.Rational(1,2), bpar: sp.Rational(1,2)}))
eq44

Eq(H'*(-6*H*r**2 + 6*H*(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2) + H'*r*(-y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2))/(4*H**2*r**3), H'**2*(-9*c_1**2*y0**2 + 4*c_1**2*(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2) + 9*c_2**2*(y1**2 + y2**2 + y3**2 + y4**2 + y5**2) - 4*c_2**2*(y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2))/(18*H**2*r**2))

In [23]:
eq44 = sp.simplify(eq44.subs({sum(y[i]**2 for i in range(6)): r_sym**2,}))
eq44

Eq(H'**2*(-y0**2 + y1**2 + y2**2 + y3**2 + y4**2 + y5**2)/(4*H**2*r**2), H'**2*(4*c_1**2*r**2 - 9*c_1**2*y0**2 - 4*c_2**2*r**2 + 9*c_2**2*(y1**2 + y2**2 + y3**2 + y4**2 + y5**2))/(18*H**2*r**2))

In [24]:
eq44 = sp.simplify(eq44.subs({sum(y[i]**2 for i in range(1,6)): r_sym**2 - y[0]**2,}))
eq44

Eq(H'**2*(r**2 - 2*y0**2)/(4*H**2*r**2), H'**2*(4*c_1**2*r**2 - 9*c_1**2*y0**2 + 5*c_2**2*r**2 - 9*c_2**2*y0**2)/(18*H**2*r**2))

We see that from the gravity equation we can impose either self-dual or anti-self-dual